In [1]:
## import libraries
from arcgis import GIS
import os
import pandas as pd
import arcpy
from arcgis.features import GeoAccessor

In [2]:
## define variables
# connect to GIS
gis = GIS()

# directory where the daily fleet csv files from GFW are stored
directory = r'F:\Global Fishing Watch\Fishing Intensity\v2\fleet-daily\fleet-daily-csvs-100-v2-2020'
processing_gdb = r'E:\analysis\GFW\gfw_daily_fleet_2020.gdb'
analysis_directory = r'E:\analysis\GFW\TIF\2020'

# To allow overwriting outputs change overwriteOutput option to True.
arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# Check out any necessary licenses.
arcpy.CheckOutExtension("spatial")
arcpy.CheckOutExtension("ImageAnalyst")

projectCoordinateSystem="GEOGCS[\"GCS_WGS_1984\",DATUM[\"D_WGS_1984\",SPHEROID[\"WGS_1984\",6378137.0,298.257223563]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]]"
gear_list = ['fishing', 'drifting_longlines', 'seiners', 'purse_seines', 'tuna_purse_seines', 'other_purse_seines', 'other_seines', 'trollers', 'fixed_gear', 'pots_and_traps', 'set_longlines', 'set_gillnets', 'dredge_fishing', 'squid_jiggers', 'other']

In [3]:
def create_directories():
    # if not arcpy.Exists(processing_gdb):
    #     arcpy.management.CreateFileGDB(os.path.dirname(processing_gdb), os.path.basename(processing_gdb))
    if not os.path.exists(analysis_directory):
        os.makedirs(analysis_directory)

In [4]:
def preprocess():
            print("Starting processing of " + filename + "...")
            # Get the full path for the filename
            file_path = os.path.join(directory, filename)
            # Read the CSV file
            print(file_path)
            global data
            data = pd.read_csv(file_path)
            # Get the first row as a table
            first_row = data.head(1)
            # Get the value in the first column
            global value
            value = first_row.iloc[0, 0]
            # replace the '-' with '_'
            value = value.replace('-', '_')
            global tif_name1
            tif_name1 = os.path.join(analysis_directory, "daily_fleet_" + value + "_hours.tif")
            global fc_hours
            fc_hours = os.path.join(processing_gdb, "daily_fleet_" + value + "_hours")
            global tif_name2
            tif_name2 = os.path.join(analysis_directory, "daily_fleet_" + value + "_fishhours.tif")
            data['cell_ll_lat'] = data['cell_ll_lat'] + 0.05
            data['cell_ll_lon'] = data['cell_ll_lon'] + 0.05
            data.rename(columns={'cell_ll_lat': 'lat'}, inplace=True)
            data.rename(columns={'cell_ll_lon': 'lon'}, inplace=True)
            print("Done with " + value + "...")


In [5]:
def process_hours():
    hours_select = data.loc[data['hours'] != 0]
    hours_csv = os.path.join(analysis_directory, "daily_fleet_" + value + "_hours.csv")
    hours_select.drop(columns=['date', 'flag', 'geartype', 'fishing_hours', 'mmsi_present'], inplace=True)
    hours_select.to_csv(hours_csv, index=False)

In [6]:
def process_fishhours():
    fishhours_select = data.loc[data['fishing_hours'] != 0]
    fishhours_csv = os.path.join(analysis_directory, "daily_fleet_" + value + "_fishhours.csv")
    fishhours_select.drop(columns=['date', 'flag', 'geartype', 'hours', 'mmsi_present'], inplace=True)
    fishhours_select.to_csv(fishhours_csv, index=False)

In [ ]:
create_directories()

for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        preprocess()
        process_hours()
        process_fishhours()
print("All done...")


In [ ]:
for filename in os.listdir(analysis_directory):
    if filename.endswith('fishhours.csv'):
        csv_name = (os.path.basename(filename))
        print("Processing " + csv_name + "...")
        fish_csv = (os.path.join(analysis_directory, os.path.splitext(filename)[0] + ".csv"))
        # print(fish_csv)
        fish_tif = (os.path.join(analysis_directory, os.path.splitext(filename)[0] + ".tif"))
        # print(fish_tif)
        fish_fc = (os.path.join(processing_gdb, os.path.splitext(os.path.basename(filename))[0]))
        # print(fish_fc)
        arcpy.management.XYTableToPoint(fish_csv, fish_fc, 'lon', 'lat', '', projectCoordinateSystem)
        arcpy.conversion.PointToRaster(fish_fc, 'fishing_hours', fish_tif, "SUM", 'fishing_hours', 0.01)

In [ ]:
for filename in os.listdir(analysis_directory):
    if filename.endswith('_hours.csv'):
        csv_name = (os.path.basename(filename))
        print("Processing " + csv_name + "...")
        hours_csv = (os.path.join(analysis_directory, os.path.splitext(filename)[0] + ".csv"))
        # print(hours_csv)
        hours_tif = (os.path.join(analysis_directory, os.path.splitext(filename)[0] + ".tif"))
        # print(hours_tif)
        hours_fc = (os.path.join(processing_gdb, os.path.splitext(os.path.basename(filename))[0]))
        # print(hours_fc)
        arcpy.management.XYTableToPoint(hours_csv, hours_fc, 'lon', 'lat', '', projectCoordinateSystem)
        arcpy.conversion.PointToRaster(hours_fc, 'hours', hours_tif, "SUM", 'hours', 0.01)

In [ ]:
## Create a mosaic dataset
mosacic_name = 'global_fish_watch_daily_fleet_sum_2000'
mosaic_path = os.path.join(processing_gdb, mosacic_name)

arcpy.management.CreateMosaicDataset(in_workspace = processing_gdb,
                                     in_mosaicdataset_name = mosacic_name,
                                     coordinate_system = projectCoordinateSystem)

In [ ]:

arcpy.management.AddRastersToMosaicDataset(in_mosaic_dataset = mosaic_path,
                                           raster_type = "Raster Dataset",
                                           input_path = analysis_directory, 
                                           sub_folder="SUBFOLDERS")


In [ ]:
arcpy.management.CalculateField(
    in_table=mosaic_path,
    field="date",
    expression="""(!Name!.split('_')[3] + "-" + !Name!.split('_')[4] + "-" + !Name!.split('_')[2])""",
    expression_type="PYTHON3",
    code_block="",
    field_type="DATE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [ ]:
arcpy.management.CalculateField(
    in_table=mosaic_path,
    field="variable",
    expression="""(!Name!.split('_')[5].split('.')[0])""",
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS")

In [ ]:
mosacic_name = 'global_fish_watch_daily_fleet_sum_2000'
mosaic_path = os.path.join(processing_gdb, mosacic_name)

arcpy.md.BuildMultidimensionalInfo(in_mosaic_dataset = mosaic_path,
                                   variable_field = 'variable',
                                   dimension_fields = 'date')

In [31]:
with arcpy.EnvManager(parallelProcessingFactor="99%"):
    arcpy.management.CalculateStatistics(in_raster_dataset = mosaic_path)

In [ ]:
with arcpy.EnvManager(compression="'LERC' 0.010000", parallelProcessingFactor="99%", pyramid="PYRAMIDS -1 NEAREST JPEG 85 NO_SKIP NO_SIPS"):
    arcpy.management.CopyRaster(
        in_raster=mosaic_path,
        out_rasterdataset=os.path.join(analysis_directory, mosacic_name + ".crf"),
        config_keyword="",
        background_value=None,
        nodata_value="",
        onebit_to_eightbit="NONE",
        colormap_to_RGB="NONE",
        pixel_type="",
        scale_pixel_value="NONE",
        RGB_to_Colormap="NONE",
        format="CRF",
        transform="NONE",
        process_as_multidimensional="ALL_SLICES",
        build_multidimensional_transpose="TRANSPOSE"
        # build_multidimensional_transpose="NO_TRANSPOSE"
    )

In [ ]:
# initial commit made